In [1]:
import torch
from utils import * 
from torch.utils.data import DataLoader
from trainer import Trainer
from transformer import TransformerDecoder
from matplotlib import pyplot as plt

set_all_seeds(42) ### DO NOT CHANGE THIS LINE
exp_name = 'case1'

train_dataset = CocoDataset(load_coco_data(max_train=1024), 'train')
train_dataloader =  DataLoader(train_dataset, batch_size=64)

val_dataset = CocoDataset(load_coco_data(max_val = 1024), 'val')
val_dataloader =  DataLoader(val_dataset, batch_size=64)

base dir  /Users/amaankazi/Library/CloudStorage/OneDrive-Personal/CMU/Fall_2024/VLR/hw3/transformers/transformer_captioning/datasets/coco_captioning
base dir  /Users/amaankazi/Library/CloudStorage/OneDrive-Personal/CMU/Fall_2024/VLR/hw3/transformers/transformer_captioning/datasets/coco_captioning


In [2]:
features, captions = next(iter(train_dataloader))[0:2]

In [3]:
transformer = TransformerDecoder(
          word_to_idx=train_dataset.data['word_to_idx'],
          idx_to_word = train_dataset.data['idx_to_word'],
          input_dim=train_dataset.data['train_features'].shape[1],
          embed_dim=256,
          num_heads=2,
          num_layers=2,
          max_length=30,
          device = 'cpu'
        )

In [4]:
logits = transformer(features, captions[:, :-1])

In [9]:
captions[:, 1:].shape

torch.Size([64, 16])

In [5]:
logits.shape

torch.Size([64, 16, 1004])

In [46]:
import torch.nn as nn
import torch.nn.functional as F

In [31]:
loss = nn.CrossEntropyLoss(reduction='none')

In [15]:
logits[0].shape

torch.Size([16, 1004])

In [39]:
null_mask = captions[:, 1:].clone()
null_mask[null_mask != 0] = 1

torch.Size([64, 16])

In [48]:
(F.cross_entropy(logits.permute(0,2,1), captions[:, 1:], reduction='none')*null_mask).sum() / null_mask.sum()

tensor(6.9891, grad_fn=<DivBackward0>)

In [37]:
pred = torch.randn(captions.shape[0], len(train_dataset.data['word_to_idx']), 16)
print(pred.shape)

torch.Size([64, 1004, 16])


In [20]:
import torch.nn as nn
import torch.nn.functional as F

In [35]:
F.cross_entropy(pred, captions[:, 1:])

RuntimeError: Expected target size [64, 1004], got [64, 16]

In [7]:
len(train_dataset.data['idx_to_word'])

1004

In [5]:
decode_captions(captions[0, 1:], train_dataset.data['idx_to_word'])

'a train at the station and people waiting <END>'

In [1]:
import torch

In [31]:
mask = torch.triu(torch.ones(3, 3, requires_grad=False), diagonal=1) != 0
# mask = mask.view(1,1,3,3)
a = torch.randn(5,4,3,3)
a.masked_fill(mask, -torch.inf)

tensor([[[[-8.1785e-01,        -inf,        -inf],
          [-1.0307e+00,  1.3950e-01,        -inf],
          [-1.9320e-01,  1.6478e+00,  2.0532e+00]],

         [[ 2.1385e+00,        -inf,        -inf],
          [-1.2400e+00,  6.0757e-01,        -inf],
          [ 1.2300e+00,  4.1352e-01, -2.0788e+00]],

         [[ 3.1992e-01,        -inf,        -inf],
          [-1.7017e+00, -5.5890e-01,        -inf],
          [-8.5494e-01,  1.7058e+00, -2.2926e+00]],

         [[ 6.2440e-01,        -inf,        -inf],
          [-1.6899e+00,  1.2521e+00,        -inf],
          [ 1.7442e-01,  2.2667e+00,  1.1365e+00]]],


        [[[-8.3528e-01,        -inf,        -inf],
          [ 9.0916e-01,  2.0245e+00,        -inf],
          [-1.5483e-01,  8.1935e-01, -6.6048e-01]],

         [[-2.5179e-02,        -inf,        -inf],
          [ 1.6319e+00,  8.0117e-02,        -inf],
          [-1.1245e+00,  5.7221e-01, -8.3986e-01]],

         [[-3.7683e-01,        -inf,        -inf],
          [ 6.643

In [13]:
torch.ones(3,3).tril()

TypeError: tril(): argument 'input' (position 1) must be Tensor, not int

In [9]:
(torch.ones(3,3)*-torch.inf).triu(1)

tensor([[0., -inf, -inf],
        [0., 0., -inf],
        [0., 0., 0.]])